In [5]:
import pandas as pd
from trakt.movies import Movie
import trakt

In [27]:
from backports import configparser

config = configparser.ConfigParser()
config.read('config.ini')

trakt.core.CLIENT_ID = config.get('TRAKT','CLIENT_ID')
trakt.core.CLIENT_SECRET = config.get('TRAKT','CLIENT_SECRET')
trakt.core.OAUTH_TOKEN = config.get('TRAKT','OAUTH_TOKEN')

In [11]:
df = pd.read_csv('export_movies_history.csv')

In [12]:
df.head()

,imdb,trakt_id,watched_at,title,year,slug
0,tt0335266,118,2020-03-10T19:09:00.000Z,b'Lost in Translation',2003.0,lost-in-translation-2003
1,tt3315342,161972,2020-03-09T20:25:00.000Z,b'Logan',2017.0,logan-2017
2,tt0305357,4757,2020-03-08T20:08:21.000Z,"b""Charlie's Angels: Full Throttle""",2003.0,charlie-s-angels-full-throttle-2003
3,tt0160127,2541,2020-03-08T20:08:12.000Z,"b""Charlie's Angels""",2000.0,charlie-s-angels-2000
4,tt3907584,221327,2020-03-08T19:59:00.000Z,b'All the Bright Places',2020.0,all-the-bright-places-2020


In [13]:
df.dtypes

imdb           object
trakt_id        int64
watched_at     object
title          object
year          float64
slug           object
dtype: object

In [14]:
df.shape

(919, 6)

In [24]:
movie1 = Movie('lost-in-translation-2003')
print(movie1.overview)
print("\n")
print(movie1.cast)
print("\n")
print(movie1.crew)
print("\n")
print(movie1.people)
movie1

Two lost souls visiting Tokyo -- the young, neglected wife of a photographer and a washed-up movie star shooting a TV commercial -- find an odd solace and pensive freedom to be real in each other's company, away from their lives in America.


[<Person>: Bill Murray, <Person>: Scarlett Johansson, <Person>: Giovanni Ribisi, <Person>: Anna Faris, <Person>: Akiko Takeshita, <Person>: Fumihiro Hayashi, <Person>: Kazuyoshi Minamimagoe, <Person>: Kazuko Shibata, <Person>: Take, <Person>: Hiroko Kawasaki, <Person>: Daikon, <Person>: Catherine Lambert, <Person>: Daiamondo Yukai, <Person>: Akiko Monô, <Person>: Françoise Dubois, <Person>: Gregory Pekar, <Person>: Ikuko Takahashi, <Person>: Osamu Shigematu, <Person>: Takashi Fujii, <Person>: Lisle Wilkerson, <Person>: Kazuo Yamada, <Person>: Akira Yamaguchi, <Person>: Diamond Yukai, <Person>: Jun Maki, <Person>: Nao Asuka, <Person>: Tetsuro Nakagawa, <Person>: Hiroshi Kawashima, <Person>: Kunichi Nomura, <Person>: Shigekazu Aida, <Person>: Kei Ta

[<Movie>: American Beauty,
 <Movie>: City of God,
 <Movie>: Unbroken,
 <Movie>: The Piano,
 <Movie>: The Edge of Seventeen,
 <Movie>: Romance,
 <Movie>: Notes on a Scandal,
 <Movie>: Certain Women,
 <Movie>: The Bridges of Madison County,
 <Movie>: 2 Days in Paris]